<a href="https://colab.research.google.com/github/Emanuel3queijos/AWS-University-ETL/blob/main/python_scripts/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicializar bibliotecas

In [ ]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt


# Scripts para adicionar data dentro do banco



In [7]:
import psycopg2

def create_connection():
    try:
        conn = psycopg2.connect(
            host="db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com",
            dbname="AWS-University-ETL-DB",
            user="postgres",
            password="HACKEIA N VEI😭)",
            port=5432
        )

        return conn
    except Exception as error:
        print(f"Erro ao conectar ao banco de dados: {error}")
        return None
create_connection()

<connection object at 0x7d541eb42700; dsn: 'user=postgres password=xxx dbname=AWS-University-ETL-DB host=db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [ ]:
def inserir_departamentos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            departamentos = [
                ("Escola de Tecnologia", "Departamento focado em áreas tecnológicas."),
                ("Escola de Engenharia", "Departamento voltado para as engenharias."),
                ("Escola de Saúde", "Departamento voltado para ciências da saúde."),
                ("Ciências Biológicas", "Departamento voltado para biologia e afins.")
            ]
            for nome, descricao in departamentos:
                cur.execute("""
                    INSERT INTO tb_departamento (nome, descricao)
                    VALUES (%s, %s)
                    RETURNING id, nome
                """, (nome, descricao))
                cod_departamento = cur.fetchone()[0]
                print(f"Departamento {nome, id} inserido com sucesso. Código: {cod_departamento}")

            conn.commit()
            print("Departamentos e inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_departamentos()


Departamento ('Escola de Tecnologia', <built-in function id>) inserido com sucesso. Código: 9
Departamento ('Escola de Engenharia', <built-in function id>) inserido com sucesso. Código: 10
Departamento ('Escola de Saúde', <built-in function id>) inserido com sucesso. Código: 11
Departamento ('Ciências Biológicas', <built-in function id>) inserido com sucesso. Código: 12
Departamentos e inseridos com sucesso.


In [ ]:
def inserir_cursos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            cursos_por_departamento = {
                "Escola de Tecnologia": [
                    ("Engenharia de Software", "Curso voltado ao desenvolvimento de software.", 4, 1200.00),
                    ("Ciência da Computação", "Foco em computação e sistemas.", 4, 1150.00),
                    ("Inteligência Artificial", "Curso para aprendizado de IA.", 4, 1300.00),
                    ("Redes de Computadores", "Infraestrutura e comunicação de redes.", 3, 1100.00)
                ],
                "Escola de Engenharia": [
                    ("Engenharia Civil", "Foco em obras e infraestrutura.", 5, 1400.00),
                    ("Engenharia Elétrica", "Curso de sistemas elétricos.", 5, 1350.00),
                    ("Engenharia Mecânica", "Curso sobre mecânica aplicada.", 5, 1380.00),
                    ("Engenharia de Controle e Automação", "Automação industrial.", 5, 1450.00)
                ],
                "Escola de Saúde": [
                    ("Medicina", "Foco em prática médica.", 6, 2000.00),
                    ("Enfermagem", "Prática de enfermagem.", 4, 1100.00),
                    ("Farmácia", "Estudos sobre medicamentos.", 4, 1250.00),
                    ("Biomedicina", "Ciência biomédica.", 4, 1200.00)
                ],
                "Ciências Biológicas": [
                    ("Biologia", "Estudo da vida.", 4, 1100.00),
                    ("Ecologia", "Curso sobre ecossistemas.", 4, 1050.00),
                    ("Zoologia", "Estudo dos animais.", 4, 1100.00),
                    ("Botânica", "Estudo das plantas.", 4, 1000.00)
                ]
            }

            for nome_departamento, cursos in cursos_por_departamento.items():
                cur.execute("""
                    SELECT id FROM tb_departamento WHERE nome = %s
                """, (nome_departamento,))
                cod_departamento = cur.fetchone()

                if cod_departamento is None:
                    print(f"Departamento '{nome_departamento}' não encontrado.")
                    continue

                cod_departamento = cod_departamento[0]

                for nome_curso, descricao_curso, duracao, mensalidade in cursos:
                    cur.execute("""
                        INSERT INTO tb_curso (nome, descricao, id_departamento, duracao, valor_mensalidade)
                        VALUES (%s, %s, %s, %s, %s)
                        RETURNING nome
                    """, (nome_curso, descricao_curso, cod_departamento, duracao, mensalidade))
                    nome = cur.fetchone()[0]
                    print(f"Curso {nome_curso} inserido com sucesso. Código:")

            conn.commit()
            print("Departamentos, cursos e disciplinas inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_cursos()


In [ ]:
# Script para inserir disciplinas e depois referencia-las aos cursos(acho q ta funfando)
from faker import Faker
import psycopg2
import random

fake = Faker('pt_BR')



def gerar_disciplinas_por_semestre():
    disciplinas = {}
    for semestre in range(1, 9):
        disciplinas[semestre] = []
        for _ in range(6):
            disciplina = {
                "nome": fake.sentence(nb_words=3),
                "creditos": fake.random_int(min=60, max=80),
                "departamento_id": random.choice([9, 10, 11]),
                "semestre": semestre
            }
            disciplinas[semestre].append(disciplina)
    return disciplinas

def inserir_disciplinas():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            for semestre, disciplinas in gerar_disciplinas_por_semestre().items():
                for disciplina_data in disciplinas:
                    nome_disciplina = disciplina_data["nome"]
                    creditos = disciplina_data["creditos"]
                    departamento = disciplina_data["departamento_id"]

                    cur.execute("""
                        INSERT INTO tb_disciplina (nome, creditos, id_departamento)
                        VALUES (%s, %s, %s)
                        RETURNING id
                    """, (nome_disciplina, creditos, departamento))
                    cod_disciplina = cur.fetchone()[0]

                    cur.execute("""
                        SELECT id FROM tb_curso WHERE id_departamento = %s
                    """, (departamento,))
                    cursos = cur.fetchall()

                    if cursos:
                        for curso in cursos:
                            cod_curso = curso[0]
                            cur.execute("""
                                INSERT INTO tb_curso_disciplina (id_curso, id_disciplina, id_departamento, semestre)
                                VALUES (%s, %s, %s, %s)
                            """, (cod_curso, cod_disciplina, departamento, semestre))

                            print(f"Disciplina '{nome_disciplina}' inserida no curso '{cod_curso}' do departamento {departamento}, semestre {semestre}.")
                    else:
                        print(f"Nenhum curso encontrado para o departamento {departamento}.")

            conn.commit()
            print("Disciplinas inseridas com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_disciplinas()


In [ ]:
from faker import Faker
import random

fake = Faker('pt_BR')
def inserir_professores():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            cur.execute("SELECT id, id_departamento FROM tb_disciplina")
            disciplinas = cur.fetchall()

            for cod_disciplina, cod_departamento in disciplinas:
                num_professores = random.randint(1, 2)

                for _ in range(num_professores):
                    nome = fake.first_name()
                    sobrenome = fake.last_name()
                    nome_completo = f"{nome}{sobrenome}"
                    matricula = fake.unique.random_number(digits=8)
                    email = f"{fake.user_name()}.{matricula}@ucsal.edu.br"
                    telefone = fake.phone_number()[:15]
                    semestre = random.randint(1, 9)
                    endereco = fake.address()
                    descricao = fake.sentence()
                    cpf = fake.unique.numerify('###########')

                    cur.execute("""
                        INSERT INTO tb_professor (nome_professor, email_professor, id_departamento, telefone, endereco, matricula, cpf)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
                        RETURNING matricula
                    """, (nome_completo, email, cod_departamento, telefone, endereco, matricula, cpf))

                    cod_professor = cur.fetchone()[0]

                    cur.execute("""
                        INSERT INTO tb_professor_disciplina (matricula_professor, id_disciplina, semestre, descricao)
                        VALUES (%s,%s,%s, %s)
                    """, (cod_professor, cod_disciplina, semestre, descricao))

                    print(f"Professor '{nome_completo}' alocado na disciplina de código '{cod_disciplina}', CPF '{cpf}', email: {email}.")

            conn.commit()
            print("Professores inseridos e alocados com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir professores: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_professores()


In [ ]:
from faker import Faker
import random
import psycopg2

fake = Faker('pt_BR')



def inserir_alunos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            conn.autocommit = False

            cur.execute("SELECT id, id_departamento, nome FROM tb_curso")
            cursos = cur.fetchall()

            alunos_batch = []
            aluno_disciplinas_batch = set()

            for _ in range(1000):
                nome_aluno = fake.unique.name()
                matricula = fake.unique.random_number(digits=8)
                email = f"{fake.user_name()}.{matricula}@ucsal.edu.br"
                cpf = fake.unique.numerify('###########')
                media_global = round(random.uniform(0, 10), 2)
                status = random.choice(['A', 'I'])

                curso = random.choice(cursos)
                id_curso, id_departamento, nome_curso = curso

                alunos_batch.append((nome_aluno, email, cpf, media_global, status, id_curso, matricula, nome_curso))
                cur.execute("SELECT id, nome FROM tb_disciplina WHERE id_departamento = %s", (id_departamento,))
                disciplinas_curso = cur.fetchall()

                for _ in range(random.randint(1, 4)):
                    if disciplinas_curso:
                        disciplina = random.choice(disciplinas_curso)
                        id_disciplina, nome_disciplina = disciplina
                        ano = random.randint(2015, 2023)
                        semestre = random.randint(1, 2)
                        nota_final = round(random.uniform(0, 10), 2)
                        status_disciplina = 'aprovado' if nota_final >= 5 else 'reprovado'
                        nota1 = round(random.uniform(0, 5), 2)
                        nota2 = round(random.uniform(0, 5), 2)

                        record = (matricula, id_disciplina, nome_disciplina, ano, semestre, nota_final, status_disciplina, nota1, nota2)
                        aluno_disciplinas_batch.add(record)

                if len(alunos_batch) >= 1000:
                    cur.executemany("""
                        INSERT INTO tb_aluno (nome_aluno, email, cpf, media_global, status, id_curso, matricula, nome_curso)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """, alunos_batch)

                    cur.executemany("""
                        INSERT INTO tb_aluno_disciplina (matricula_aluno, id_disciplina, nome_disciplina, ano, semestre, nota_final, status_disciplina, nota1, nota2)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, list(aluno_disciplinas_batch))

                    print(f"Batch de {len(alunos_batch)} alunos inseridos com sucesso.")
                    conn.commit()
                    alunos_batch.clear()
                    aluno_disciplinas_batch.clear()

            if alunos_batch:
                cur.executemany("""
                    INSERT INTO tb_aluno (nome_aluno, email, cpf, media_global, status, id_curso, matricula, nome_curso)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, alunos_batch)

            if aluno_disciplinas_batch:
                cur.executemany("""
                    INSERT INTO tb_aluno_disciplina (matricula_aluno, id_disciplina, nome_disciplina, ano, semestre, nota_final, status_disciplina, nota1, nota2)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, list(aluno_disciplinas_batch))

            conn.commit()
            print("Alunos e associações inseridos com sucesso.")
        except Exception as error:
            print(f"Erro ao inserir alunos: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_alunos()


## Importar CSVS

In [14]:
import psycopg2
import os

def export_to_csv():

    try:
        conn = create_connection()
        cur = conn.cursor()
        tables = ["tb_aluno", "tb_curso", "tb_disciplina", "tb_professor"]
        for table in tables:
            with open(f'{table}.csv', 'w', newline='') as f:
                cur.copy_expert(f"COPY {table} TO STDOUT WITH CSV HEADER", f)

        cur.close()
        print("Dados exportados com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao exportar dados: {error}")
    finally:
        if conn is not None:
            cur.close()
            conn.close()

export_to_csv()


Dados exportados com sucesso.
